In [1]:
import pandas as pd
DR = pd.read_csv('DR.csv')

def UpdateList(L):
    return L/max(L)

PostTagList = ['CC','CD','DT','EX','FW','IN','JJ','JJR','JJS','LS','MD','NN','NNS','NNP','NNPS','PDT','POS','PRP','PRP1','RB','RBR','RBS','RP','TO','UH','VB','VBD','VBG','VBN','VBZ','WDT','WP','WP1','WRB']

print DR.columns
print DR.dtypes

for PS in PostTagList:
    print PS
    print max(DR[PS])
    DR[PS] = UpdateList(DR[PS])
    print max(DR[PS])
    print ""
    


Index([u'Unnamed: 0', u'Unnamed: 0.1', u'Comment', u'Duration', u'FDA_Alerts',
       u'Drug_Class', u'Drug_Name', u'UserRating', u'Useful_Reviews', u'Date',
       u'NReviews', u'Drug_Interactions', u'Condition', u'Major', u'Moderate',
       u'Minor', u'DurationCategory', u'CC', u'CD', u'DT', u'EX', u'FW', u'IN',
       u'JJ', u'JJR', u'JJS', u'LS', u'MD', u'NN', u'NNS', u'NNP', u'NNPS',
       u'PDT', u'POS', u'PRP', u'PRP1', u'RB', u'RBR', u'RBS', u'RP', u'TO',
       u'UH', u'VB', u'VBD', u'VBG', u'VBN', u'VBP', u'VBZ', u'WDT', u'WP',
       u'WP1', u'WRB', u'Polarity', u'Subjectivity'],
      dtype='object')
Unnamed: 0             int64
Unnamed: 0.1           int64
Comment               object
Duration              object
FDA_Alerts           float64
Drug_Class            object
Drug_Name             object
UserRating           float64
Useful_Reviews        object
Date                  object
NReviews               int64
Drug_Interactions     object
Condition             object
M

In [2]:
#Drop Rows that Have an Error in the 'Useful_Reviews'
inx = []

for i in range(0,len(DR)):
    try:
        k = float(DR.iloc[i,8])
    except:
        inx.append(i)
        
        
DR = DR.drop(DR.index[inx])

In [3]:
#Drop the first two columns from DR and assign the data frame to DR_Final
DR_Final = DR.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1)

# print DR_Final.shape
# print DR_Final.columns

In [ ]:
print len(DR_Final)
print len(inx)

In [4]:
#Coerce 'Useful_Reviews' to float
DR_Final['Useful_Reviews'] = pd.to_numeric(DR_Final['Useful_Reviews'],errors='coerce')

#DR_Sub for Correlation_Matrix
DR_Sub = DR_Final.iloc[:,[2,5,6,8,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,38,39,40,41,42,43,44,45,46,47,49,50,51]]


In [6]:
# Visualize Correlation_Matrix from DR_Sub
from pandas import set_option
import matplotlib.pyplot as plt
import numpy as np
%matplotlib tk

set_option('precision',1)
# print DR_New.corr(method='pearson')

fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(DR_Sub.corr(),vmin=-1,vmax=1,interpolation='none')
fig.colorbar(cax)
ticks = np.arange(0,44,1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
plt.show()


In [ ]:
print DR_Sub.columns
print DR_Sub.dtypes

In [7]:
#Model-1: Predict the Major Interactions (NOT YET COMPLETE)
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline



#The Features for DR_Sub_X are constructed from ALL of the POSTags
# DR_Sub_X = DR_Final.iloc[:,[2,5,6,8,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,38,39,40,41,42,43,44,45,46,47,49,50,51]]

#The Features for DR_Sub_X from POSTag Counting are NOT Considered
DR_Sub_X = DR_Final.iloc[:,[2,5,6,8,12,13,14]]

# DR_Sub_X = DR_Final.iloc[:,[2,6,8,12,13,14,15,20,21,26,28,32,34,35,36,37,50,51]]

#Use column '5' to predict 'UserRating' and column '11' to predict 'Major Drug Interactions'
DR_Sub_Y = DR_Final.iloc[:,11]


#Calculate the (column) average of the matrices
DRSubXMean = DR_Sub_X.mean()
DRSubYMean = DR_Sub_Y.mean()

#Fill the missing 'NA' values with the respective column average
for i in range(0,len(DRSubXMean)):
    DR_Sub_X[DR_Sub_X.columns[i]] = DR_Sub_X[DR_Sub_X.columns[i]].fillna(DRSubXMean[i])
    
DR_Sub_Y = DR_Sub_Y.fillna(DRSubYMean)
    
DR_Sub_X = DR_Sub_X.values
DR_Sub_Y = DR_Sub_Y.values

validation_size = 0.2
seed = 11

X_train, X_validation,Y_train,Y_validation = train_test_split(DR_Sub_X,DR_Sub_Y,test_size=validation_size,random_state=seed)

#Spotcheck Algorithms
num_folds = 10
scoring = 'neg_mean_squared_error'

models = []
models.append(('LR1',LinearRegression()))
models.append(('LASSO',Lasso()))
models.append(('EN',ElasticNet()))
models.append(('KNN',KNeighborsRegressor()))
models.append(('CART',DecisionTreeRegressor()))
# models.append(('SVR',SVR()))

results = []
names = []
print 'ok'
for name,model in models:
#     print name, model
    kfold = KFold(n_splits=num_folds,random_state=seed)
    cv_results = cross_val_score(model,X_train,Y_train,cv=kfold,scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(),cv_results.std())
    print msg

    
print ""
print ""


    

/home/krishnaveni/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


ok
LR1: -11788.851902 (141.989698)
LASSO: -11789.206083 (141.869877)
EN: -11795.254989 (140.124445)
KNN: -192.939819 (44.553414)
CART: -6.160186 (4.963956)




In [8]:
#Standardize the Data
pipelines = []

pipelines.append(('ScaledLR',Pipeline([('Scaler',StandardScaler()),('LR',LinearRegression())])))
pipelines.append(('ScaledLASSO',Pipeline([('Scaler',StandardScaler()),('LASSO',Lasso())])))
pipelines.append(('ScaledEN',Pipeline([('Scaler',StandardScaler()),('EN',ElasticNet())])))
# pipelines.append(('ScaledKNN',Pipeline([('Scaler',StandardScaler()),('KNN',KNeighborsRegressor())])))
pipelines.append(('ScaledCART',Pipeline([('Scaler',StandardScaler()),('CART',DecisionTreeRegressor())])))

results = []
names = []
print 'ok'
for name,model in pipelines:
    kfold = KFold(n_splits=num_folds,random_state=seed)
    cv_results = cross_val_score(model,X_train,Y_train,cv=kfold,scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(),cv_results.std())
    print msg
    
print 'ok1'

ok
ScaledLR: -11788.851902 (141.989698)
ScaledLASSO: -11794.243203 (140.986427)
ScaledEN: -12199.006648 (129.293949)
ScaledCART: -4.710475 (3.777197)
ok1


In [9]:
#Ensemble Methods
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor

ensembles = []
ensembles.append((' ScaledAB',Pipeline([('Scaler',StandardScaler()),('AB',AdaBoostRegressor())])))
# ensembles.append(('ScaledGBM',Pipeline([('Scaler',StandardScaler()),('GBM',GradientBoostingRegressor())])))
ensembles.append(('ScaledRF',Pipeline([('Scaler',StandardScaler()),('RF',RandomForestRegressor())])))
ensembles.append(('ScaledET',Pipeline([('Scaler',StandardScaler()),('ET',ExtraTreesRegressor())])))

seed = 7

results = []
names = []
for name, model in ensembles:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

 ScaledAB: -9546.929232 (547.368227)


/home/krishnaveni/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ScaledRF: -6.382952 (6.459032)
ScaledET: -3.488218 (3.398077)


In [ ]:
print max(DR_Final['Useful_Reviews'])